# Module Imports

In [1]:
## IMPORT MODULES
import pandas as pd
import numpy as np
from sklearn import linear_model
from scipy.interpolate import griddata
import scipy.odr


In [2]:
## IMPORT PLOTTING MODULES
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib as mpl
from matplotlib import cm
%matplotlib widget

# Dataframe

In [3]:
## FULL DATAFRAME
df_raw = pd.read_csv("WS_2-Policy_Summary.csv")

## DROP EXTRANEOUS DATAPOINTS
# df_raw = df_raw.drop(df_raw[(df_raw['landing_rate']<=0.15) & (df_raw['vz_d']>=2.5)].index)
# df_raw = df_raw.drop(df_raw[(df_raw['landing_rate']<=0.15) & (df_raw['vx_d']>=1.0)].index)
df_raw = df_raw.drop(df_raw[df_raw['My_d']<-7.7].index)




## GROUP RAW DATA BY IC AND FILTER BY MAX LANDING RATE
df_raw_max = df_raw.groupby(['vz_d','vx_d']).agg({'landing_rate':"max"}).reset_index()
idx = df_raw.groupby(['vz_d','vx_d'])['landing_rate'].transform(max) == df_raw['landing_rate']
df_max = df_raw[idx].reset_index()


## CLEAN RAW DATA
df_max['My_d'] = df_max['My_d'].apply(lambda x: np.abs(x)) # Convert My to magnitude
df_max['My_d'] = df_max['My_d'].apply(lambda x: 7.7 if x>7.7 else x) # Cap My_d to max possible moment
df_max = df_max.dropna()



In [4]:
# TEMPLATE FOR LINEAR REGRESSION
from sklearn.linear_model import LinearRegression

def linreg(X ,Y):
    # perform multivariate or univariate
    # X = [var1 var2]
    # y = var3
    reg = LinearRegression(normalize=True)
    reg.fit(X,Y)

    intercept = reg.intercept_ 
    coef = reg.coef_
    
    R2 = reg.score(X,Y) # built in function for r^2
    
    ypred = reg.predict(X) # plot x vs. ypred for line or plane
    
    return ypred,intercept,coef,R2


# Landing Rate Data

In [5]:
## LANDING RATE DATA (RAW)
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# DEFINE VARIABLES
X = df_max['vx_d']
Y = df_max['vz_d']
Z = df_max['landing_rate']


# CREATE PLOTS AND COLOR BAR
ax.scatter(X,Y,Z)


# PLOT LIMITS AND INFO
ax.set_zlim(0,1)

ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('Landing Rate (%)')
ax.set_title('Optimal Policy Landing Rate')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
## AVG LANDING RATE SURFACE
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# DEFINE VARIABLES
X = df_max['vx_d']
Y = df_max['vz_d']
Z = df_max['landing_rate']

# DEFINE COLOR FORMATS AND LIMITS
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=1)

# CREATE PLOTS AND COLOR BAR
ax.plot_trisurf(X,Y,Z,cmap = cmap,norm=norm,edgecolors='grey',linewidth=0.25)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate (%)")


# PLOT LIMITS AND INFO
ax.set_zlim(0,1)

ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('Landing Rate (%)')
ax.set_title('Landing Rate (Raw Data)')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
## AVG LANDING RATE SMOOTHED SURFACE
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# DEFINE VARIABLES
X = df_max['vx_d']
Y = df_max['vz_d']
Z = df_max['landing_rate']

# SOMETHING ABOUT DEFINING A GRID
xi = np.linspace(X.min(),X.max(),(len(Z)//3))
yi = np.linspace(Y.min(),Y.max(),(len(Z)//3))
zi = griddata((X, Y), Z, (xi[None,:], yi[:,None]), method='linear')
xig, yig = np.meshgrid(xi, yi)

# DEFINE COLOR FORMAT AND LIMITS
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=1)

# CREATE PLOTS AND COLOR BAR
surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm,edgecolors='grey',linewidth=0.25)
# surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm)
# surf = ax.contour(xig, yig, zi,cmap=cmap,norm=norm)

fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate (%)")

# PLOT LIMITS AND INFO
ax.set_zlim(0,1)

ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('Landing Rate (%)')
ax.set_title('Landing Rate (Raw Data)')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# RREV vs IC

In [8]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")




# LINEAR REGRESSION
X_reg = df_temp[['vx_d','vz_d']]
Y_reg = df_temp['RREV_threshold']

ypred, intercept,coef,R2 = linreg(X_reg,Y_reg)
print(f"Equation: RREV_reg = {intercept:.2f} + {coef[0]:.2f}*vx_d + {coef[1]:.2f}*vz_d \n")
print(f"R^2 = {R2:.3f}")


# DEFINE VARIABLES
X = df_temp['vx_d']
Y = df_temp['vz_d']
Z = df_temp['RREV_threshold']
C = df_temp['RREV_threshold']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=3.5,vmax=5.5)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm,zorder=2)
ax.plot_trisurf(X,Y,ypred,alpha=0.75,label='Linear_Regression',zorder=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV_threshold (1/s)")



# PLOT LIMITS AND INFO
ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.5)
ax.set_zlim(3,6)

ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('RREV_threshold (1/s)')
ax.set_title('RREV_thr vs IC - (Optimal Data)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Equation: RREV_reg = 5.51 + 0.00*vx_d + -0.35*vz_d 

R^2 = 0.590


In [9]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")


# DEFINE VARIABLES
X = df_temp['vx_d']
Y = df_temp['vz_d']
Z = df_temp['RREV_threshold']

# SOMETHING ABOUT DEFINING A GRID
xi = np.linspace(X.min(),X.max(),(len(Z)//10))
yi = np.linspace(Y.min(),Y.max(),(len(Z)//10))
zi = griddata((X, Y), Z, (xi[None,:], yi[:,None]), method='linear')
xig, yig = np.meshgrid(xi, yi)

# DEFINE COLOR FORMAT AND LIMITS
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=3.5,vmax=5.5)

# CREATE PLOTS AND COLOR BAR
surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm,edgecolors='grey',linewidth=0.25)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV_threshold")

# PLOT LIMITS AND INFO
ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.5)
ax.set_zlim(3,6)

ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('RREV_threshold (1/s)')

ax.set_title('Optimal RREV_thr vs IC')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Rotation Time Data

In [10]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")

# DEFINE VARIABLES
X = df_temp['RREV_threshold']
Y = df_temp['OF_y']
Z = df_temp['impact_tdelta']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
ax.scatter(X,Y,Z)

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate %)")



# PLOT LIMITS AND INFO
ax.set_xlabel('RREV (1/s)')
ax.set_ylabel('OF_y (rad/s)')
ax.set_zlabel('Delta_t (s)')

ax.set_title('Time Rotating vs IC (Optimal Data)')

ax.set_zlim(0,0.5)




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# My_d vs IC Data

In [11]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")

# DEFINE VARIABLES
X = df_temp['vx_d']
Y = df_temp['vz_d']
Z = df_temp['My_d']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
ax.scatter(X,Y,Z)

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate %)")



# PLOT LIMITS AND INFO
ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('My_d (N*mm)')

ax.set_title('My_d vs IC (Optimal Data)')

ax.set_zlim(0,10)




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Impact Angle Data

In [12]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")


# DEFINE VARIABLES
X = df_temp['vx_d']
Y = df_temp['vz_d']
Z = -df_temp['impact_eul']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('Impact Angle (deg)')

ax.set_title('Impact Angle vs IC (Optimal Data)')

ax.set_zlim(60,180)
ax.set_zticks([60,90,120,150,180,210])



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")


# DEFINE VARIABLES
X = df_temp['vx_d']
Y = df_temp['vz_d']
Z = -df_temp['impact_eul']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=60,vmax=180)
ax.plot_trisurf(X,Y,Z,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('Impact Angle (deg)')

ax.set_title('Impact Angle vs IC (Optimal Data)')

ax.set_zlim(60,180)
ax.set_zticks([60,90,120,150,180,210])



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
## AVG LANDING RATE SMOOTHED SURFACE
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")

# DEFINE VARIABLES
X = df_temp['vx_d']
Y = df_temp['vz_d']
Z = -df_temp['impact_eul']

# SOMETHING ABOUT DEFINING A GRID
xi = np.linspace(X.min(),X.max(),(len(Z)//10))
yi = np.linspace(Y.min(),Y.max(),(len(Z)//10))
zi = griddata((X, Y), Z, (xi[None,:], yi[:,None]), method='linear')
xig, yig = np.meshgrid(xi, yi)

# DEFINE COLOR FORMAT AND LIMITS
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=60,vmax=180)

# CREATE PLOTS AND COLOR BAR
surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm,edgecolors='grey',linewidth=0.25)
# surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm)
# surf = ax.contour(xig, yig, zi,cmap=cmap,norm=norm)

fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Impact Angle (deg)")

# PLOT LIMITS AND INFO
ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('Impact Angle (deg)')

ax.set_title('Impact Angle vs IC (Optimal Data)')

ax.set_zlim(60,180)
ax.set_zticks([60,90,120,150,180,210])


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Moment vs RREV

In [15]:
fig = plt.figure()
ax = fig.add_subplot(111)

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")

# DEFINE VARIABLES
X = df_temp['RREV_threshold']
Y = df_temp['My_d']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.Greys
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate (%)")



# PLOT LIMITS AND INFO
ax.set_xlabel('RREV_threshold (1/s)')
ax.set_ylabel('My_d (N*mm)')
ax.set_title('Policy: My_d vs RREV_thr')


ax.grid()
ax.set_xlim(2,7)
ax.set_ylim(0,10)
ax.hlines(7.77,2,7)
ax.text(2.05,8.1,'Motors Maxed Out \n(My_d = 7.7 N*mm)')




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
fig = plt.figure()
ax = fig.add_subplot(111)

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")

# DEFINE VARIABLES
X = df_temp['RREV_threshold']
Y = df_temp['My_d']
C = df_temp['vz_d']

# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=1.5,vmax=4)
ax.scatter(X,Y,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Z-Velocity (m/s)")



# PLOT LIMITS AND INFO
ax.set_xlabel('RREV_threshold (1/s)')
ax.set_ylabel('My_d (N*mm)')
ax.set_title('Policy: My_d vs RREV_thr')




ax.grid()
ax.set_xlim(2,7)
ax.set_ylim(0,10)
ax.hlines(7.77,2,7)
ax.text(2.05,8.1,'Motors Maxed Out \n(My_d = 7.7 N*mm)')




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# POLICY FITTING

## RAW POLICY RELATION

In [17]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.6}")



# DEFINE VARIABLES
X = df_temp['RREV_threshold']
Y = df_temp['OF_y']
Z = df_temp['My_d']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV (1/s)')
ax.set_ylabel('OF_y (rad/s)')
ax.set_zlabel('My_d (N*mm)')

ax.set_title('Raw Policy Relation')



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# LINEAR MODEL FIT

## Regression with y-based loss

In [18]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_reg = df_max.query(f"landing_rate>={0.6}")


# LINEAR REGRESSION
X_reg = df_reg[['RREV_threshold','OF_y']]
Y_reg = df_reg['My_d']

ypred, intercept,coef,R2 = linreg(X_reg,Y_reg)
print(f"Equation: My_d = {intercept:.2f} + {coef[0]:.2f}*RREV + {coef[1]:.2f}*OF_y \n")
print(f"R^2 = {R2:.3f}")



# DEFINE VARIABLES
X = df_max['RREV_threshold']
Y = df_max['OF_y']
Z = df_max['My_d']
C = df_max['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.plot_trisurf(X_reg['RREV_threshold'],X_reg['OF_y'],ypred,alpha=0.75,label='Linear_Regression',zorder=1) # Surface plot
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm) # Scatter plot
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV (1/s)')
ax.set_ylabel('OF_y (rad/s)')
ax.set_zlabel('My_d (N*mm)')

ax.set_title('Optimal Policy Relation \n(Z-Based Error)')



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Equation: My_d = -5.21 + 2.32*RREV + 0.40*OF_y 

R^2 = 0.173


## Regression with x-based loss


In [19]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_reg = df_max.query(f"landing_rate>={0.6}")


# LINEAR REGRESSION
X_reg = df_reg[['My_d','OF_y']]
Y_reg = df_reg['RREV_threshold']

ypred, intercept,coef,R2 = linreg(X_reg,Y_reg)
print(f"Equation: RREV_thr = {intercept:.2f} + {coef[0]:.2f}*My_d + {coef[1]:.2f}*OF_y \n")
print(f"Equation: My_d = ((RREV_thr - {intercept:.2f}) - {coef[1]:.2f}*OF_y)/{coef[0]:.2f})")
print(f"R^2 = {R2:.3f}")



# DEFINE VARIABLES
X = df_max['RREV_threshold']
Y = df_max['OF_y']
Z = df_max['My_d']
C = df_max['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.plot_trisurf(ypred,X_reg['OF_y'],X_reg['My_d'],alpha=0.75,label='Linear_Regression',zorder=1) # Surface plot
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm) # Scatter plot
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV (1/s)')
ax.set_ylabel('OF_y (rad/s)')
ax.set_zlabel('My_d (N*mm)')

ax.set_title('Optimal Policy Relation \n(X-Based Error)')



plt.show()




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Equation: RREV_thr = 3.88 + 0.07*My_d + -0.12*OF_y 

Equation: My_d = ((RREV_thr - 3.88) - -0.12*OF_y)/0.07)
R^2 = 0.431


# Regression w/ Orthogonal Distance

In [20]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_reg = df_max.query(f"landing_rate>={0.6}")


# LINEAR REGRESSION
def linfit(beta, x):
    return beta[0]*x[0] + beta[1]*x[1] + beta[2] 

x1 = df_reg['RREV_threshold']
x2 = df_reg['OF_y']
y = df_reg['My_d']
x = np.row_stack( (x1, x2) ) #odr doesn't seem to work with column_stack

linmod = scipy.odr.Model(linfit)
data = scipy.odr.Data(x, y)
odrfit = scipy.odr.ODR(data, linmod, beta0=[1., 1., 1.])
odrres = odrfit.run()
odrres.pprint()


y_reg = odrres.beta[0]*x1 + odrres.beta[1]*x2 + odrres.beta[2]

print(f"Equation: My_d = {odrres.beta[2]:.2f} + {odrres.beta[0]:.2f}*RREV + {odrres.beta[1]:.2f}*OF_y \n")
# print(f"R^2 = {R2:.3f}")



# DEFINE VARIABLES
X = df_max['RREV_threshold']
Y = df_max['OF_y']
Z = df_max['My_d']
C = df_max['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.plot_trisurf(x1,x2,y_reg,alpha=0.75,label='Linear_Regression',zorder=1) # Surface plot
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm) # Scatter plot
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate (%)")



# PLOT LIMITS AND INFO
ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV (1/s)')
ax.set_ylabel('OF_y (rad/s)')
ax.set_zlabel('My_d (N*mm)')

ax.set_title('Optimal Policy Relation \n(Orthogonal Regression)')



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Beta: [ 14.24775773   1.80019726 -54.95025462]
Beta Std Error: [ 3.3874206   0.45801538 14.15137079]
Beta Covariance: [[ 156.13758091   18.29283331 -651.16969919]
 [  18.29283331    2.8544952   -74.05045078]
 [-651.16969919  -74.05045078 2724.99819251]]
Residual Variance: 0.07349043227476758
Inverse Condition #: 0.004802836564341187
Reason(s) for Halting:
  Sum of squares convergence
Equation: My_d = -54.95 + 14.25*RREV + 1.80*OF_y 



In [21]:
import numpy as np
import scipy.odr

n = 1000
t = np.linspace(0, 1, n)

def linfit(beta, x):
    return beta[0]*x[0] + beta[1]*x[1] + beta[2] #notice changed indices for x

x1 = 2.5*np.sin(2*np.pi*6*t)+4
x2 = 0.5*np.sin(2*np.pi*7*t + np.pi/3)+2

x = np.row_stack( (x1, x2) ) #odr doesn't seem to work with column_stack

e = 0.25*np.random.randn(n)
y = 3*x[0] + 4*x[1] + 5 + e #indices changed

linmod = scipy.odr.Model(linfit)
data = scipy.odr.Data(x, y)
odrfit = scipy.odr.ODR(data, linmod, beta0=[1., 1., 1.])
odrres = odrfit.run()
odrres.pprint()


Beta: [3.00250336 4.08324528 4.83154598]
Beta Std Error: [0.0044915  0.02257085 0.0492668 ]
Beta Covariance: [[ 8.55088631e-03  7.47701194e-05 -3.43531103e-02]
 [ 7.47701194e-05  2.15935140e-01 -4.32262762e-01]
 [-3.43531103e-02 -4.32262762e-01  1.02881281e+00]]
Residual Variance: 0.0023592412703615476
Inverse Condition #: 0.030352551306084316
Reason(s) for Halting:
  Sum of squares convergence


In [22]:
odrres.beta

array([3.00250336, 4.08324528, 4.83154598])

In [23]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
y_reg = odrres.beta[0]*x1 + odrres.beta[1]*x2 + odrres.beta[2]

ax.scatter(x1,x2,y)
ax.plot_trisurf(x1,x2,y_reg)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …